In [43]:
import numpy as np
import pandas as pd
import os
import re
from datetime import date
import sqlalchemy
import sys
import mariadb
import pymysql

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [44]:
# set file name want to process
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'

In [23]:
# database connection with mariaDB for read the read and write data from and to dataframe <----> mariaDB tables
database_username = 'root'
database_password = 'password'
database_ip       = '127.0.0.1:3306'
database_name     = 'data_dashboard'
database_connection = sqlalchemy.create_engine('mariadb+mariadbconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [4]:
# fetch data from tbl_unit to data frame
tbl_unit = pd.read_sql('SELECT unit_id, unit_name FROM tbl_unit', database_connection)
tbl_unit.head(5)

,unit_id,unit_name
0,2,FFU
1,8,FPC11
2,10,FPC12
3,6,FPC13
4,1,FPC14


In [5]:
# fetch data from tbl_log_parameters to data frame
tbl_parameter = pd.read_sql('SELECT prm_id, prm_name FROM tbl_log_parameters', database_connection)
tbl_parameter.head(5)

,prm_id,prm_name
0,1,Arduino
1,101,Arduino1
2,66,Arduino1_LT41
3,125,Arduino1_LT42
4,126,Arduino1_LT43


## Data Overview

In [6]:
log_rawDF = pd.read_csv(file_name, sep='*')
log_rawDF.head(5)

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
5,2021-4-20 13:3:46,1618916626,209680,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerd...",NaN,NaN
6,2021-4-20 13:3:58,1618916638,271269,3,FPC14,program,step ID - Changed (new and old value),4_WaterAcid,5_empty
7,2021-4-20 13:4:1,1618916641,101959,3,FPC14,program,start at - Changed (new and old value),sellected step at...,first step
8,2021-4-20 13:4:6,1618916646,386245,3,FPC14,program,procedure running - Changed,not active,active
9,2021-4-20 13:4:7,1618916647,639887,2,FPC14,program,Program F_ThalaPseu_HalfNutr started,NaN,NaN


In [8]:
log_rawDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 865668 entries, 0 to 865667
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Date_Time  865668 non-null  object
 1   Time       865668 non-null  int64 
 2   MicroSec   865668 non-null  int64 
 3   Type       865668 non-null  int64 
 4   Unit       865668 non-null  object
 5   Parameter  864368 non-null  object
 6   Message    865668 non-null  object
 7   Value1     411877 non-null  object
 8   Value2     495869 non-null  object
dtypes: int64(3), object(6)
memory usage: 59.4+ MB


## Replace Parameter values and unit values with Table int values

In [15]:
# data type conversion and droping rows if parameter column value is missing
def log_tweak(log_rawDF):
    return (
    log_rawDF
    .query('Type != 1')
    .query('Unit == ("FPC11", "FPC12", "FPC13", "FPC14", "FPC21", "FPC22", "FPC23", "FPC24", "FFU", "FPP", "Product", "scheduler")')
    .dropna(subset=['Parameter'])
    .drop(columns=['MicroSec'])
    .assign(Unit = log_rawDF.Unit.map(tbl_unit.set_index('unit_name')['unit_id']),
           Parameter = log_rawDF.Parameter.map(tbl_parameter.set_index('prm_name')['prm_id']),
           Date_Time = pd.to_datetime(log_rawDF.Date_Time)
           )
    .astype({'Type' : 'int8', 'Unit' : 'int16', 'Parameter' : 'int32'})
    .rename(columns = {'Date_Time':'date_time', 'Time':'time_epoch', 'Type':'category', 'Unit':'unit', 'Parameter':'parameter', 'Message':'message', 'Value1': 'new_value', 'Value2':'old_value'})
    .fillna('empty')
    #.sort_values('Time')
    #.replace('/'',' ', regex=True)
    #.info()
    )

log_cleaned = log_tweak(log_rawDF)

In [16]:
print(log_cleaned.info())

Data frame Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 570819 entries, 6 to 863803
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date_time   570819 non-null  datetime64[ns]
 1   time_epoch  570819 non-null  int64         
 2   category    570819 non-null  int8          
 3   unit        570819 non-null  int16         
 4   parameter   570819 non-null  int32         
 5   message     570819 non-null  object        
 6   new_value   570819 non-null  object        
 7   old_value   570819 non-null  object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(1), int8(1), object(3)
memory usage: 29.9+ MB
None

Nullvalues in each columns
date_time     0
time_epoch    0
category      0
unit          0
parameter     0
message       0
new_value     0
old_value     0
dtype: int64

Lists of unique values in unit column
[ 1  2  3  4  5  6  7  8  9 10 11]

Total number of unique values in uni

## Export Log Data into tbl_log_data

In [17]:
connection = pymysql.connect(host='localhost',
                            user='root',
                            password='password',
                            db='data_dashboard')

    # create cursor
cursor=connection.cursor()

In [18]:
cols = "`,`".join([str(i) for i in log_cleaned.columns.tolist()])
cols

'date_time`,`time_epoch`,`category`,`unit`,`parameter`,`message`,`new_value`,`old_value'

In [19]:
log_cleaned.shape

(570819, 8)

In [20]:
for i,row in log_cleaned.iterrows():
    sql = "INSERT INTO `tbl_log_data` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))
    # the connection is not autocommitted by default, so we must commit to save our changes
    #connection.commit()

In [21]:
connection.commit()
connection.close()

In [24]:
tbl_logdata = pd.read_sql('SELECT * FROM tbl_log_data', database_connection)
tbl_logdata.head(10)

In [39]:
log_cleaned.head(10)

,date_time,time_epoch,category,unit,parameter,message,new_value,old_value
6,2021-04-20 13:03:58,1618916638,3,1,2,step ID - Changed (new and old value),4_WaterAcid,5_empty
7,2021-04-20 13:04:01,1618916641,3,1,2,start at - Changed (new and old value),sellected step at...,first step
8,2021-04-20 13:04:06,1618916646,3,1,2,procedure running - Changed,not active,active
9,2021-04-20 13:04:07,1618916647,2,1,2,Program F_ThalaPseu_HalfNutr started,empty,empty
10,2021-04-20 13:04:07,1618916647,2,1,2,Step 4_WaterAcid started,empty,empty
11,2021-04-20 13:04:10,1618916650,3,1,2,procedure running - Changed,active,not active
12,2021-04-20 13:04:11,1618916651,2,1,2,_======program F_ThalaPseu_HalfNutr stopped be...,empty,empty
13,2021-04-20 13:04:23,1618916663,3,1,2,start at - Changed (new and old value),sellected step at...,first step
14,2021-04-20 13:04:23,1618916663,3,1,2,procedure running - Changed,not active,active
15,2021-04-20 13:04:25,1618916665,2,1,2,Program F_ThalaPseu_HalfNutr started,empty,empty


In [33]:
tbl_logdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570819 entries, 0 to 570818
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date_time   570819 non-null  datetime64[ns]
 1   time_epoch  570819 non-null  int64         
 2   category    570819 non-null  int64         
 3   unit        570819 non-null  int64         
 4   parameter   570819 non-null  int64         
 5   message     570819 non-null  object        
 6   new_value   570819 non-null  object        
 7   old_value   570819 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 34.8+ MB


In [40]:
log_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570819 entries, 6 to 863803
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date_time   570819 non-null  datetime64[ns]
 1   time_epoch  570819 non-null  int64         
 2   category    570819 non-null  int8          
 3   unit        570819 non-null  int16         
 4   parameter   570819 non-null  int32         
 5   message     570819 non-null  object        
 6   new_value   570819 non-null  object        
 7   old_value   570819 non-null  object        
dtypes: datetime64[ns](1), int16(1), int32(1), int64(1), int8(1), object(3)
memory usage: 46.1+ MB


## Appendix

In [42]:
#tbl_logdata.compare(log_cleaned)

In [35]:
merged = tbl_logdata.merge(log_cleaned, indicator=True, how='outer')
merged[merged['_merge'] == 'right_only']

,date_time,time_epoch,category,unit,parameter,message,new_value,old_value,_merge


In [36]:
merged = log_cleaned.merge(tbl_logdata, indicator=True, how='outer')
merged[merged['_merge'] == 'right_only']

,date_time,time_epoch,category,unit,parameter,message,new_value,old_value,_merge


In [ ]:
'''
command = '''DROP TABLE IF EXISTS localytics_app2;
CREATE TABLE localytics_app2
(
"Index" serial primary key,
"Event" text,
"Day" timestamp without time zone,
);'''
cursor.execute(command)
connection.commit()
'''

SyntaxError: invalid syntax (4013186024.py, line 2)

In [ ]:
'''
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():
     cursor.execute("INSERT INTO HumanResources.DepartmentTest (DepartmentID,Name,GroupName) values(?,?,?)", row.DepartmentID, row.Name, row.GroupName)
cnxn.commit()
cursor.close()
'''

'\n# Insert Dataframe into SQL Server:\nfor index, row in df.iterrows():\n     cursor.execute("INSERT INTO HumanResources.DepartmentTest (DepartmentID,Name,GroupName) values(?,?,?)", row.DepartmentID, row.Name, row.GroupName)\ncnxn.commit()\ncursor.close()\n'